**Instalar**

url api: https://open-meteo.com/en/docs/historical-weather-api#start_date=2022-01-01&end_date=2022-12-31

**Instalar el APi's**

pip install openmeteo-requests                                                                                                                                         
pip install requests-cache retry-requests numpy pandas

In [5]:
#Importar
import requests
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Configura el cliente de la API Open-Meteo con caché y reintento en caso de error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


In [2]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "2022-01-01",
	"end_date": "2022-12-31",
	"hourly": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s"),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 52.5°E 13.400009155273438°N
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                    date  temperature_2m
0    2022-01-01 00:00:00       11.238999
1    2022-01-01 01:00:00       11.089000
2    2022-01-01 02:00:00       11.089000
3    2022-01-01 03:00:00       11.188999
4    2022-01-01 04:00:00       11.089000
...                  ...             ...
8755 2022-12-31 19:00:00       14.889000
8756 2022-12-31 20:00:00       14.589000
8757 2022-12-31 21:00:00       14.639000
8758 2022-12-31 22:00:00       14.988999
8759 2022-12-31 23:00:00       14.988999

[8760 rows x 2 columns]
